# cypher query graphDB

In [1]:
#graphDB config

NEO4J_URI="neo4j+s://3d5ed496.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="nacJCOrhkUV5oXAL55FxnyvEzQjN63vAKqIEMNViHH4"

In [9]:
import os
import dotenv
os.environ['NEO4J_URI']=NEO4J_URI
os.environ['NEO4J_USERNAME']=NEO4J_USERNAME
os.environ['NEO4J_PASSWORD']=NEO4J_PASSWORD

In [6]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [11]:
dotenv.load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [12]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    groq_api_key=groq_api_key,
    model_name='Gemma2-9b-it'
    )
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E073D6B700>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E073D77400>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk(born June 28, 1971) is a businessman and investor known 
for his key roles in the space company SpaceX and the automotive company 
Tesla, Inc. Other involvements include ownership of X Corp., the company 
that operates the social media platform X (formerly known as Twitter), 
and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. 
He is one of the wealthiest individuals in the world; as of August 2024 
Forbes estimates his net worth to be US$247 billion. Musk was born in Pretoria, 
South Africa, to Maye (née Haldeman), a model, and Errol Musk, a businessman 
and engineer. Musk briefly attended the University of Pretoria before immigrating 
to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. 
"""
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content='\nElon Reeve Musk(born June 28, 1971) is a businessman and investor known \nfor his key roles in the space company SpaceX and the automotive company \nTesla, Inc. Other involvements include ownership of X Corp., the company \nthat operates the social media platform X (formerly known as Twitter), \nand his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. \nHe is one of the wealthiest individuals in the world; as of August 2024 \nForbes estimates his net worth to be US$247 billion. Musk was born in Pretoria, \nSouth Africa, to Maye (née Haldeman), a model, and Errol Musk, a businessman \nand engineer. Musk briefly attended the University of Pretoria before immigrating \nto Canada at the age of 18, acquiring citizenship through his Canadian-born mother. \n')]

In [17]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [18]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [20]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye Haldeman', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='University Of Pretoria', type='Institution', properties={}), Node(id='Canada', type='Country', properties={})], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye Haldeman', type='Person', properties={}), type='BORN_TO', properties={}), Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='BORN_TO', properties={}), Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='University Of Pretoria', type='Institution', properties={}), type='ATTENDED', properties={}), Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Canada', type='Country', properties={}), type='BEC

In [21]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye Haldeman', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='University Of Pretoria', type='Institution', properties={}),
 Node(id='Canada', type='Country', properties={})]

In [22]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye Haldeman', type='Person', properties={}), type='BORN_TO', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='BORN_TO', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='University Of Pretoria', type='Institution', properties={}), type='ATTENDED', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Canada', type='Country', properties={}), type='BECAME_CITIZEN', properties={})]

# load dataset to graphDB and generating cypher queries

In [53]:
# load the movie dataset 

movie_query=""" 
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released=date(row.released),
    m.title=row.title,
    m.imdbRating=tofloat(row.imdbRating)
FOREACH (director in split(row.director,'|')|
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]-> (m)
         )
FOREACH (actor in split(row.actors,'|')|
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]-> (m)
        )
FOREACH (genre in split(row.genres,'|')|
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENERE]->(g)
        )

"""

In [54]:
graph

In [55]:
# insert records 
graph.query(movie_query)

[]

In [56]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {title: STRING, released: DATE, id: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENERE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


# create cypher query using llm model

In [57]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001E0736CD700>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E073D6B700>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E073D77400>, model_name=

In [60]:
response=chain.invoke({'query':'who were the actors and directors of the movie GoldenEye'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:ACTED_IN|DIRECTED]-(p:Person) RETURN p.name AS personName 

Full Context:
[{'personName': 'Pierce Brosnan'}, {'personName': 'Famke Janssen'}, {'personName': 'Sean Bean'}, {'personName': 'Izabella Scorupco'}, {'personName': 'Martin Campbell'}]

> Finished chain.


{'query': 'who were the actors and directors of the movie GoldenEye',
 'result': 'Pierce Brosnan, Famke Janssen, Sean Bean, Izabella Scorupco and Martin Campbell  \n'}